In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)


Last run:  2021-10-28 02:55:50.658807


In [16]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import pandas as pd
import sys,json
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

#https://w.wiki/4J4B
queryLittNoPic = """SELECT ?item ?itemLabel (year(?born) as ?year) ?statements ?Litteraturbankenid
 (CONCAT(?itemLabel," ",str(year(?born))) AS ?search) WHERE {
   ?item wdt:P5101 ?Litteraturbankenid ;
         wikibase:statements ?statements .
 # FILTER (?statements <100)
  minus {?item wdt:P18 ?img}
  OPTIONAL{?item wdt:P569 ?born}
  FILTER (year(?born) < 1900)
  FILTER (year(?born) > 1800)
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}  order by desc(?statements)
"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)


def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDLittNoPic = get_sparql_dataframe(endpoint_url, queryLittNoPic)
WDLittNoPic.shape

(334, 6)

In [17]:
WDLittNoPic

,item,itemLabel,year,statements,Litteraturbankenid,search
0,http://www.wikidata.org/entity/Q4935182,Ruth Almén,1870,51,AlménR,Ruth Almén 1870
1,http://www.wikidata.org/entity/Q20652547,Franz Hoffmann,1814,50,HoffmannF,Franz Hoffmann 1814
2,http://www.wikidata.org/entity/Q3427152,Emmy Köhler,1858,49,KöhlerE,Emmy Köhler 1858
3,http://www.wikidata.org/entity/Q1744240,Laura Kieler,1849,48,KielerL,Laura Kieler 1849
4,http://www.wikidata.org/entity/Q5913839,Rudolf Klinckowström,1816,45,KlinckowströmRM,Rudolf Klinckowström 1816
...,...,...,...,...,...,...
329,http://www.wikidata.org/entity/Q52635024,Ludvig Alexander Rosencrantz Balchen,1830,6,BalchenLAR,Ludvig Alexander Rosencrantz Balchen 1830
330,http://www.wikidata.org/entity/Q52690406,J.E. Ahlstrand,1814,5,AhlstrandJE,J.E. Ahlstrand 1814
331,http://www.wikidata.org/entity/Q100605054,Heinrich Ortenburg,1850,5,OrtenburgH,Heinrich Ortenburg 1850
332,http://www.wikidata.org/entity/Q52577083,Gunnar Wetterberg,1863,5,WetterbergG,Gunnar Wetterberg 1863


In [26]:
import urllib3, json
http = urllib3.PoolManager() 
SPAdetail = "https://portrattarkiv.se/details/"
url= "https://portrattarkiv.se/endpoints/search.php"  
urlbasePic = "https://portrattarkiv.se/endpoints/file.php?id="

for index, row  in WDLittNoPic.iterrows():
    print("\n",row["search"],row["item"])
    encoded_body = json.dumps({
        "limit": "5",
        "from": "0",
        "birthyear":row["year"],
        "all":row["search"]
    })
    
    r = http.request('POST', url,
                 headers={'Content-Type': 'application/json'},
                 body=encoded_body)

    if r.status != 200:
        print(r.status)
        continue
    
    data = json.loads(r.data.decode('utf-8'),) 
    urls = []


    for h in data["hits"]["hits"]:
        id = h["_id"]
        #print(h)
        source = h["_source"]
        try:
            urlPicture = urlbasePic + id
#            urls.append(urlPicture)
            score = h["_score"]
            FirstName = source["FirstName"]
            LastName = source["LastName"]
            BirthYear = source["BirthYear"]
            if score > 30:
                print("\t\t",score,FirstName, " ", LastName, " - ", BirthYear,SPAdetail+id, "\t", )
        except:
            print("Error")


 Ruth Almén 1870 http://www.wikidata.org/entity/Q4935182
		 31.814905 Ruth   Almén  -  1870 https://portrattarkiv.se/details/28K7pO-SRrAAAAAAAACjdQ 	
		 31.549816 Ruth   Almén  -  1870 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACaGDg 	
		 30.749416 Rut   Almén  -  1870 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABOZRg 	

 Franz Hoffmann 1814 http://www.wikidata.org/entity/Q20652547

 Emmy Köhler 1858 http://www.wikidata.org/entity/Q3427152

 Laura Kieler 1849 http://www.wikidata.org/entity/Q1744240

 Rudolf Klinckowström 1816 http://www.wikidata.org/entity/Q5913839
		 38.487625 Rudolf   Klinckowström  -  1816 https://portrattarkiv.se/details/mLh5P0pzYFAAAAAAAAAuNQ 	
		 34.456703 Rudolf   Klinckowström  -  1816 https://portrattarkiv.se/details/0piq-fX0h5AAAAAAAABVlA 	
		 33.157272 Rudolf   Klinckowström  -  1816 https://portrattarkiv.se/details/0piq-fX0h5AAAAAAAAA5tA 	
		 32.378304 Rudolf   Klinckowström  -  1816 https://portrattarkiv.se/details/WQknIBFpuuAAAAAAAAN9hA 	

 

		 35.845913 Ragnar   Pihlstrand  -  1850 https://portrattarkiv.se/details/Dysd0OxEWfAAAAAAAABGew 	
		 34.49644 Ragnar   Pihlstrand  -  1850 https://portrattarkiv.se/details/Dysd0OxEWfAAAAAAAABGfA 	

 Carl Santesson 1885 http://www.wikidata.org/entity/Q16650285

 Ezaline Boheman 1863 http://www.wikidata.org/entity/Q19976305
		 45.646877 Ezaline   Boheman  -  1863 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABlWOA 	
		 40.269062 Ezaline   Boheman  -  1863 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA2H2g 	

 Reinhold Ericson 1845 http://www.wikidata.org/entity/Q23989269

 Karl Strecker 1862 http://www.wikidata.org/entity/Q53954436

 Emily Jane Harding 1850 http://www.wikidata.org/entity/Q56480189

 Emil Jacob Jonas 1824 http://www.wikidata.org/entity/Q4205935

 Julius Regis 1889 http://www.wikidata.org/entity/Q6066281

 Olof Fryxell 1806 http://www.wikidata.org/entity/Q5746206
		 34.245766 Olof   Fryxell  -  1806 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAB2_RQ 	
		 31


 Carl August Olausson 1865 http://www.wikidata.org/entity/Q108378867

 Hjalmar Wernberg 1867 http://www.wikidata.org/entity/Q108441194

 Hjalmar Gustaf Ferdinand Höglund 1865 http://www.wikidata.org/entity/Q108641487

 Gustaf Carlsson Leijonhufvud 1861 http://www.wikidata.org/entity/Q108649719

 Lars Jesper Benzelstierna 1808 http://www.wikidata.org/entity/Q5570937

 Bror Edvard Fornell 1820 http://www.wikidata.org/entity/Q5736382
		 44.82057 Bror   Fornell  -  1820 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABLhDA 	
		 42.961197 Bror   Fornell  -  1820 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABhUJg 	
		 39.049408 Bror   Fornell  -  1820 https://portrattarkiv.se/details/IQojCnw0WmAAAAAAAAAbYw 	
		 38.27113 Bror   Fornell  -  1820 https://portrattarkiv.se/details/IQojCnw0WmAAAAAAAABO7Q 	
		 37.585163 Bror   Fornell  -  1820 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAB2_bg 	

 Beata Sofia Gyldén 1804 http://www.wikidata.org/entity/Q11853991

 Maria Nyström 1854 htt


 Carl Bernhard Rydström 1819 http://www.wikidata.org/entity/Q6085748
		 32.808388 Carl   Rydström  -  1819 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACbk_Q 	

 Alfred Brynolf Roos 1824 http://www.wikidata.org/entity/Q11850819

 Hugo von Becker 1834 http://www.wikidata.org/entity/Q11863739

 Ines Lindström 1855 http://www.wikidata.org/entity/Q11864837

 Lydia Blom 1853 http://www.wikidata.org/entity/Q11879928

 Johan Henrik Chronwall 1851 http://www.wikidata.org/entity/Q15740252

 Eva Agatha Stanislas Barnekow 1813 http://www.wikidata.org/entity/Q52554783

 Olof Abraham Ericsson 1822 http://www.wikidata.org/entity/Q108916413

 Johan Niclas Holmgren 1802 http://www.wikidata.org/entity/Q95326825

 Carl Larsson 1889 http://www.wikidata.org/entity/Q99756355

 Anders Sundler 1822 http://www.wikidata.org/entity/Q106397191

 Frithiof Strömmerstedt 1863 http://www.wikidata.org/entity/Q107490955

 Ottilia Westermark 1835 http://www.wikidata.org/entity/Q108374435

 Carl Lagergren 1830 htt


 Gustaf Göran Stenborg 1832 http://www.wikidata.org/entity/Q108896522

 Augusta Bergh 1869 http://www.wikidata.org/entity/Q98545788

 Karin Anderson 1875 http://www.wikidata.org/entity/Q99929803

 Mathilda Ahlbom 1834 http://www.wikidata.org/entity/Q106594297

 Djos Per Andersson 1814 http://www.wikidata.org/entity/Q5554362

 Hans Andersson 1817 http://www.wikidata.org/entity/Q108826166

 Erik Westholm 1816 http://www.wikidata.org/entity/Q108835875

 Gustaf Georg Silfversparre 1846 http://www.wikidata.org/entity/Q108896667

 Carl Henrik Rydberg 1820 http://www.wikidata.org/entity/Q108919118

 Jon. Ad. Bolm 1811 http://www.wikidata.org/entity/Q108920733

 Theodor Giertz 1833 http://www.wikidata.org/entity/Q108587959
		 30.790638 Theodor   Giertz  -  1833 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABye3g 	

 Charlotta Tunelli 1829 http://www.wikidata.org/entity/Q108825047

 Erik Högström 1820 http://www.wikidata.org/entity/Q108869019
		 31.745737 Erik   Högström  -  1820 https://p